In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/MachineLearningRating_v3.txt',sep="|")

# Preview the data
df.head()

C:\Users\user\AppData\Local\Temp\ipykernel_9012\121985174.py:1: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.txt',sep="|")


,UnderwrittenCoverID,PolicyID,TransactionMonth,IsVATRegistered,Citizenship,LegalType,Title,Language,Bank,AccountType,...,ExcessSelected,CoverCategory,CoverType,CoverGroup,Section,Product,StatutoryClass,StatutoryRiskType,TotalPremium,TotalClaims
0,145249,12827,2015-03-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
1,145249,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,21.929825,0.0
2,145249,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Windscreen,Windscreen,Windscreen,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0
3,145255,12827,2015-05-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,512.848070,0.0
4,145255,12827,2015-07-01 00:00:00,True,,Close Corporation,Mr,English,First National Bank,Current account,...,Mobility - Metered Taxis - R2000,Own damage,Own Damage,Comprehensive - Taxi,Motor Comprehensive,Mobility Metered Taxis: Monthly,Commercial,IFRS Constant,0.000000,0.0


In [4]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency, ttest_ind, f_oneway

# Create derived columns
df['HasClaim'] = (df['TotalClaims'] > 0).astype(int)
df['Margin'] = df['TotalPremium'] - df['TotalClaims']


In [6]:
# Create a contingency table
province_claim_table = pd.crosstab(df['Province'], df['HasClaim'])

# Chi-square test
chi2, p, dof, ex = chi2_contingency(province_claim_table)

print(f"H₀: No risk difference across provinces\nChi2 = {chi2:.2f}, p-value = {p}")
if p < 0.05:
    print("Reject H₀: There IS a significant difference in claim frequency across provinces.")
else:
    print("Fail to reject H₀: No significant difference in claim frequency across provinces.")


H₀: No risk difference across provinces
Chi2 = 104.19, p-value = 5.925510718204678e-19
Reject H₀: There IS a significant difference in claim frequency across provinces.


In [10]:
# Use only top 5 postal codes to reduce noise/sparsity
top_postals = df['PostalCode'].value_counts().head(5).index
filtered_df = df[df['PostalCode'].isin(top_postals)]

postal_claim_table = pd.crosstab(filtered_df['PostalCode'], filtered_df['HasClaim'])
chi2, p, dof, ex = chi2_contingency(postal_claim_table)

print(f"\nH₀: No risk difference across postal codes\nChi2 = {chi2:.2f}, p-value = {p}")
if p < 0.05:
    print("Reject H₀: Risk **differs** across postal codes.")
else:
    print("Fail to reject H₀: No significant risk difference across postal codes.")



H₀: No risk difference across postal codes
Chi2 = 60.23, p-value = 2.5997263446847924e-12
Reject H₀: Risk **differs** across postal codes.


In [11]:
# Pick the top 2 most common postal codes for comparison
postal1, postal2 = top_postals[:2]
group1 = filtered_df[filtered_df['PostalCode'] == postal1]['Margin']
group2 = filtered_df[filtered_df['PostalCode'] == postal2]['Margin']

t_stat, p = ttest_ind(group1, group2, equal_var=False)

print(f"\nH₀: No margin difference between PostalCode {postal1} and {postal2}\nt-stat = {t_stat:.2f}, p-value = {p}")
if p < 0.05:
    print("Reject H₀: There **is** a significant margin difference.")
else:
    print("Fail to reject H₀: No significant margin difference.")



H₀: No margin difference between PostalCode 2000 and 122
t-stat = 1.16, p-value = 0.24446241842452004
Fail to reject H₀: No significant margin difference.


In [8]:
gender_claim_table = pd.crosstab(df['Gender'], df['HasClaim'])
chi2, p, dof, ex = chi2_contingency(gender_claim_table)

print(f"\nH₀: No risk difference between Women and Men\nChi2 = {chi2:.2f}, p-value = {p:.5f}")
if p < 0.05:
    print("Reject H₀: There IS a significant difference in risk between Women and Men.")
else:
    print("Fail to reject H₀: No significant difference in risk between Women and Men.")




H₀: No risk difference between Women and Men
Chi2 = 7.26, p-value = 0.02657
Reject H₀: There IS a significant difference in risk between Women and Men.
